In [4]:
# import dependencies
import pandas as pd
# from geopy.geocoders import Nominatim
import requests
import json
import gmaps
from pandas import json_normalize
from config import g_key

In [5]:
# Read in dataset
housing_data = pd.read_csv("Resources/Clean/San_Diego_Housing_Data.csv", index_col=0)
# Reset index to add back zipcodes
housing_data.reset_index(inplace=True)

# Make new df with San Diego zipcodes
zipcodesdf = housing_data[["Zip Code"]]
zipcodesdf

,Zip Code
0,91901
1,91902
2,91906
3,91910
4,91911
...,...
80,92131
81,92139
82,92154
83,92173


In [6]:
# Read in US zipcode database
us_zip = pd.read_csv("Resources/RAW/uszips.csv")

In [7]:
# Create dataframe for just zipcodes and lat/lon
lat_lng = us_zip[['zip', 'lat', 'lng']]
# Rename the zip column to zipcode so we can merge
lat_lng.rename(columns={'zip':'Zip Code'}, inplace=True)
lat_lng.head()

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_61107/3011836463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lat_lng.rename(columns={'zip':'Zip Code'}, inplace=True)


,Zip Code,lat,lng
0,601,18.18027,-66.75266
1,602,18.36075,-67.17541
2,603,18.45744,-67.12225
3,606,18.16585,-66.93716
4,610,18.29110,-67.12243


In [8]:
# Merge dataframes on the zipcode
zip_lat = zipcodesdf.merge(lat_lng, on='Zip Code')
zip_lat.head(20)

,Zip Code,lat,lng
0,91901,32.81467,-116.71929
1,91902,32.67479,-117.00475
2,91906,32.66241,-116.47255
3,91910,32.63653,-117.06301
4,91911,32.60686,-117.04984
5,91913,32.62174,-116.98609
6,91914,32.66604,-116.95426
7,91915,32.62266,-116.95013
8,91932,32.56923,-117.11798
9,91935,32.70214,-116.78721


In [9]:
# Bring the index backto zipcode so we can iterate through safely.
zip_lat.set_index(['Zip Code'], inplace=True)

In [10]:
# Setting our API parameters
params = {
    "radius": 5000,
    "type": 'supermarket',
    "key": g_key
}

In [11]:
# Iterate the responses and filter out market names in a zipcode.
market_list = []

for index, row in zip_lat.iterrows():
    lat = zip_lat.loc[index, 'lat']
    lng = zip_lat.loc[index, 'lng']
    
    
    params["location"] = f"{lat},{lng}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    markets = requests.get(base_url,params=params).json()

    market_list.extend(markets.get('results'))


In [12]:
# Confirm we got results
len(market_list)

516

In [13]:
# Confirm we got coordinates for each market in a zipcode
market_list[0]


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 32.6389997, 'lng': -117.050924},
  'viewport': {'northeast': {'lat': 32.64043683029151,
    'lng': -117.0494107197085},
   'southwest': {'lat': 32.63773886970851, 'lng': -117.0521086802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png',
 'icon_background_color': '#4B96F3',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/shoppingcart_pinlet',
 'name': 'Smart & Final Extra!',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 4032,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110969422095860249667">Jorge Gonzalez</a>'],
   'photo_reference': 'AcYSjRhSG0TWcLI4sKabGcQMs7vhm8opHYJPu4W9HLHsTzZvAv-v-oLh28geR6lmtQJn4wzFUT8JuUt-rRSkJKfwf6djM0KszRDr6Ox0kS2SGa6VpL3s7qzEGy6nKR1oLYwhtSMVfkPWDv_NiqWunI7rIf-Y07t0d43yEhwFaiHvd7Grmp0N',
   'width': 3024}],
 'place_id': 'ChIJC0Yh2z5O2YARzpeuMePXZ9g',
 'plus_code': {'compound_co

In [14]:
# Pull business operation status from our JSON
df2 = pd.DataFrame([x['business_status'] for x in market_list])
df2.set_axis(['business_status'], axis='columns', inplace=True)
df2

,business_status
0,OPERATIONAL
1,OPERATIONAL
2,OPERATIONAL
3,OPERATIONAL
4,OPERATIONAL
...,...
511,OPERATIONAL
512,OPERATIONAL
513,CLOSED_TEMPORARILY
514,OPERATIONAL


In [15]:
# Pull market names from our JSON
df3 = pd.DataFrame([x['name'] for x in market_list])
df3.set_axis(['name'],axis='columns', inplace=True)
df3

,name
0,Smart & Final Extra!
1,Sprouts Farmers Market
2,Carnival Supermarket
3,Target Grocery
4,99 Ranch Market
...,...
511,Abarrotes La Tepexpense
512,Waldo's
513,Farmamil chaparral
514,Abarrotes La Placita


In [16]:
# Pull lat/lon from our JSON
df4 = pd.DataFrame([x['geometry']['location'] for x in market_list])
df4

,lat,lng
0,32.639000,-117.050924
1,32.627807,-117.074994
2,32.621859,-117.072892
3,32.653616,-117.065593
4,32.629342,-117.040931
...,...,...
511,32.538416,-117.066559
512,32.534882,-117.040981
513,32.540419,-117.031336
514,32.535808,-117.037852


In [17]:
# Merge all our dataframes into one
final_market_df = pd.concat([df2, df3, df4], axis = 1)
final_market_df

,business_status,name,lat,lng
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892
3,OPERATIONAL,Target Grocery,32.653616,-117.065593
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931
...,...,...,...,...
511,OPERATIONAL,Abarrotes La Tepexpense,32.538416,-117.066559
512,OPERATIONAL,Waldo's,32.534882,-117.040981
513,CLOSED_TEMPORARILY,Farmamil chaparral,32.540419,-117.031336
514,OPERATIONAL,Abarrotes La Placita,32.535808,-117.037852


In [19]:
# Getting zipcodes for each market
import geopy

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']


geolocator = geopy.Nominatim(user_agent='user-agent')

zipcodes = final_market_df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='lat', lon_field='lng')


In [20]:
# Export dataframe as js file 
output_file = 'markets.js'
zipcodes.to_json(output_file, orient='records')

In [21]:
# Reading the js file and exporting to dataframe
zipsjs = pd.read_json('./markets.js')
zipsjs = pd.DataFrame(zipsjs)
zipsjs

,shop,house_number,road,city,county,state,ISO3166-2-lvl4,postcode,country,country_code,...,highway,village,state_district,commercial,district,residential,tourism,place,office,quarter
0,Smart & Final Extra!,360.0,East H Street,Chula Vista,San Diego County,California,US-CA,91910.0,United States,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sprouts Farmers Market,690.0,3rd Avenue,Chula Vista,San Diego County,California,US-CA,91910.0,United States,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Carnival,870.0,3rd Avenue,Chula Vista,San Diego County,California,US-CA,91911.0,United States,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Ring Road,NaN,San Diego County,California,US-CA,91910.0,United States,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99 Ranch Market,505.0,Telegraph Canyon Road,Chula Vista,San Diego County,California,US-CA,91910.0,United States,us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,NaN,NaN,Avenida Mar Blanco,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22534.0,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,Plaza de La Mujer,NaN,Calle Benito Juárez,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22010.0,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zona Río
513,NaN,NaN,Calle Larroque,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22010.0,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zona Río
514,NaN,NaN,Plaza Santa Cecilia,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22050.0,México,mx,...,NaN,NaN,NaN,NaN,NaN,NaN,Hotel Nelson,NaN,NaN,NaN


In [22]:
# Checking the columns of the new dataframe
zipsjs.columns

Index(['shop', 'house_number', 'road', 'city', 'county', 'state',
       'ISO3166-2-lvl4', 'postcode', 'country', 'country_code', 'amenity',
       'retail', 'town', 'hamlet', 'building', 'neighbourhood', 'suburb',
       'highway', 'village', 'state_district', 'commercial', 'district',
       'residential', 'tourism', 'place', 'office', 'quarter'],
      dtype='object')

In [23]:
# Keeping relevant columns
markets_zipcodes = zipsjs[['postcode', 'country_code']]
markets_zipcodes = pd.DataFrame(markets_zipcodes)

In [24]:
# Merge all our dataframes into one
final_market_df = pd.concat([df2, df3, df4, markets_zipcodes], axis = 1)
final_market_df

,business_status,name,lat,lng,postcode,country_code
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924,91910.0,us
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994,91910.0,us
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892,91911.0,us
3,OPERATIONAL,Target Grocery,32.653616,-117.065593,91910.0,us
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931,91910.0,us
...,...,...,...,...,...,...
511,OPERATIONAL,Abarrotes La Tepexpense,32.538416,-117.066559,22534.0,mx
512,OPERATIONAL,Waldo's,32.534882,-117.040981,22010.0,mx
513,CLOSED_TEMPORARILY,Farmamil chaparral,32.540419,-117.031336,22010.0,mx
514,OPERATIONAL,Abarrotes La Placita,32.535808,-117.037852,22050.0,mx


In [25]:
# Removing markets that are not operational
final_market_df = final_market_df[final_market_df['business_status'] == 'OPERATIONAL']
final_market_df

,business_status,name,lat,lng,postcode,country_code
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924,91910.0,us
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994,91910.0,us
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892,91911.0,us
3,OPERATIONAL,Target Grocery,32.653616,-117.065593,91910.0,us
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931,91910.0,us
...,...,...,...,...,...,...
510,OPERATIONAL,Soriana Super,32.539066,-117.066646,22050.0,mx
511,OPERATIONAL,Abarrotes La Tepexpense,32.538416,-117.066559,22534.0,mx
512,OPERATIONAL,Waldo's,32.534882,-117.040981,22010.0,mx
514,OPERATIONAL,Abarrotes La Placita,32.535808,-117.037852,22050.0,mx


In [26]:
# Removing NaN or other non-numeric values from the postcode column
final_market_df = final_market_df[pd.to_numeric(final_market_df['postcode'], errors='coerce').notnull()]
final_market_df

,business_status,name,lat,lng,postcode,country_code
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924,91910.0,us
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994,91910.0,us
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892,91911.0,us
3,OPERATIONAL,Target Grocery,32.653616,-117.065593,91910.0,us
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931,91910.0,us
...,...,...,...,...,...,...
508,OPERATIONAL,Super 777,32.538053,-117.039070,22050.0,mx
510,OPERATIONAL,Soriana Super,32.539066,-117.066646,22050.0,mx
511,OPERATIONAL,Abarrotes La Tepexpense,32.538416,-117.066559,22534.0,mx
512,OPERATIONAL,Waldo's,32.534882,-117.040981,22010.0,mx


In [27]:
final_market_df.postcode = final_market_df.postcode.apply(int)
final_market_df

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_61107/1126803790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_market_df.postcode = final_market_df.postcode.apply(int)


,business_status,name,lat,lng,postcode,country_code
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924,91910,us
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994,91910,us
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892,91911,us
3,OPERATIONAL,Target Grocery,32.653616,-117.065593,91910,us
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931,91910,us
...,...,...,...,...,...,...
508,OPERATIONAL,Super 777,32.538053,-117.039070,22050,mx
510,OPERATIONAL,Soriana Super,32.539066,-117.066646,22050,mx
511,OPERATIONAL,Abarrotes La Tepexpense,32.538416,-117.066559,22534,mx
512,OPERATIONAL,Waldo's,32.534882,-117.040981,22010,mx


In [28]:
final_market_df = final_market_df[final_market_df['country_code'] == 'us']
final_market_df

,business_status,name,lat,lng,postcode,country_code
0,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924,91910,us
1,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994,91910,us
2,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892,91911,us
3,OPERATIONAL,Target Grocery,32.653616,-117.065593,91910,us
4,OPERATIONAL,99 Ranch Market,32.629342,-117.040931,91910,us
...,...,...,...,...,...,...
495,OPERATIONAL,El Picador Foods,32.573865,-117.055439,92154,us
496,OPERATIONAL,Liva Distributor,32.568932,-117.064476,92173,us
497,OPERATIONAL,La Bodega Market,32.552844,-117.042370,92173,us
498,OPERATIONAL,Pancho Villa Mercado,32.561686,-117.063112,92173,us


In [29]:
# exporting file
from pathlib import Path

filepath = Path("Resources/Clean/San_Diego_Markets.csv")
final_market_df.to_csv(filepath)